In [18]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import sys
import os
src_path='../src'
if not src_path in sys.path:
    sys.path.insert(1,src_path)
from preposessing import *
from utils_OB import *
from _tracking import *
%load_ext autoreload
import pickle
%autoreload 2
import pandas as pd
from utils_JK import *
from _postprocessing import *
import _generate_time_series
import seaborn as sns
sns.set_theme(context='paper', style = 'ticks', color_codes='deep')
matplotlib.use('module://matplotlib_inline.backend_inline')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
folder_path='../../../5hz_resampled_int_trans_clustered_3/'
omegas,phis,names=import_folder_for_tracking(folder_path)
freqs=radtoHz(omegas)

In [20]:
'../../'

'../../'

In [21]:
# FAVARElli initial

def Favarelli_initial(F_all ):
    Nt = 200
    N_lim = 50
    Bf = 0.01
    epsilon = 0.2
    sf = 0.01
    #Divide data to initial phase and online phase
    F_initial = F_all[:,:Nt]
    F_online = F_all[:,Nt:]

    #Number of bins
    Nbins = int(np.ceil((F_initial.max() - F_initial.min())/Bf))

    #Make a histogram of the 
    hist, bin_edges = np.histogram(F_initial, bins = Nbins, density = False)


    #Extract the initial frequencies
    id = np.array([i for i, v in enumerate(hist >= N_lim) if v]) #Find the highest peaks in the histogram

    #Find the bin edges for each of the detected initial frequency bins and reshape for easier for-loop
    found_bin_edges = np.array([])
    for i in id:
        found_bin_edges = np.append(found_bin_edges, np.array([bin_edges[i], bin_edges[i+1]]))
    found_bin_edges = found_bin_edges.reshape((int(found_bin_edges.shape[0]/2), 2))

    F_initial_flat = F_initial.flatten()

    f0 = np.array([])

    for f_start, f_end in found_bin_edges:
        id_f = np.where(np.logical_and(F_initial_flat <= f_end, F_initial_flat >= f_start)) #Indices of frequencies in the bin
        f0 = np.append(f0, np.mean(F_initial_flat[id_f])) #Mean of the frequencies in the bin
    return f0
    

In [22]:
suggested_initials=[]
suggested_split_initials=[]

initil_segs=[]


for i in range(0,len(freqs)-100,5):
    
    initial_freqs=freqs[i:i+100]
    initial_phis=phis[i:i+100]
    # non splitting
    initial_traces=get_initial_modes(initial_freqs,initial_phis)
    suggested_initials.append(initial_traces)
    
    #splitting 
    initial_traces=get_initial_modes2(initial_freqs,initial_phis)
    suggested_split_initials.append(initial_traces)   
    
    #first segment_only
    temp_segs=[]
    for i in range(len(initial_freqs[0])):
        temp_segs.append(ModeTrace(initial_freqs[0][i],initial_phis[0][i],0))
    initil_segs.append(temp_segs)
    

    


In [23]:

with open('suggested_initials_100.pkl', 'wb') as fout:
    pickle.dump(suggested_initials, fout)


with open('suggested_split_initials_100.pkl', 'wb') as fout:
    pickle.dump(suggested_split_initials, fout)

with open('initial_segs_50.pkl', 'wb') as fout:
    pickle.dump(initil_segs, fout)

In [24]:
segments=[]
folder_path='../../../5hz_resampled_int_trans_clustered_3/'

for name in os.listdir(folder_path):
    segments.append(import_converted_ts(folder_path,name))






In [25]:
print(len(segments))

3853


In [26]:
lambds=[]
for seg in segments:
    lambds.append(np.array(seg.lambds_n_auto)) 
input=_generate_time_series.create_output_guassian(lambds)
freqs=input.T**2

C:\Users\olebfa\AppData\Local\Temp\ipykernel_15380\3761380342.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  lambds.append(np.array(seg.lambds_n_auto))
c:\Users\olebfa\OneDrive - NTNU\00Master - Onedrive\OB ymse\Remote_desktop\modal-tracking-thesis-spring-2023\Postprocessing\../src\_generate_time_series.py:287: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ys)


In [27]:
print(len(freqs[5]))

1300


In [28]:
favarelli_freqs=[]
for i in range(0,len(freqs)-201,50):
    input2=_generate_time_series.create_output_guassian(lambds)
    freqs=input2.T**2
    # print(freqs)
    favarelli_freqs.append(Favarelli_initial(freqs[i:i+201]))

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
Favarelli_full=[]

Nt = 200
N_lim = 50
Bf = 0.01
epsilon = 0.2
sf = 0.01
    
for i in range(0,len(input)-200,30):
    Favarelli_full.append(perform_tracking_favarelli(freqs[i:i+200],Nt=Nt,N_lim=N_lim,Bf=Bf,epsilon=epsilon,sf=sf))

IndexError: index 0 is out of bounds for axis 1 with size 0

In [ ]:
for seg in Favarelli_full:
    print(len(seg))

2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
0
0
1
0
1
0
0
1
0
0
0
0
1
0
0
1
0
1
1
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:

with open('favarelli_freqs.pkl', 'wb') as fout:
    pickle.dump(favarelli_freqs, fout)

In [ ]:
with open('test.pkl', "rb") as file:
    test2= pickle.load(file)
print(test
      )

[<_tracking.ModeTrace object at 0x000001B29AA63370>, <_tracking.ModeTrace object at 0x000001B29AA63130>]


In [ ]:
with open('initial_segs.pkl', "rb") as file:
    initial_segs= pickle.load(file)
    
with open('suggested_initials.pkl', "rb") as file:
    sug_segs= pickle.load(file)
    
with open('suggested_split_initials.pkl', "rb") as file:
    sug_split_segs= pickle.load(file)
    
with open('favarelli_freqs.pkl', "rb") as file:
    initial_favarelli= pickle.load(file)

In [ ]:
print(len(initial_segs))
print(len(sug_segs))
print(len(sug_split_segs))
print(len(initial_favarelli))

78
78
78
0


2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
0
0
1
0
1
0
0
1
0
0
0
0
1
0
0
1
0
1
1
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
